## 30. 条件付きGAN(CGAN)
条件付きGANとは、生成器と識別器にいくつかの追加情報を与えて、条件付きができるように訓練を行う敵対的生成ネットワークである。  
追加情報は、ラベルでもタグや言葉でもよいが、ここではラベルを用いる。  
CGANの生成器はリアルな画像を生成するだけではなく、ラベルにもマッチしていなければならない。  
これを訓練させれば、望みのラベルを与えることでCGANに生成してほしいサンプルの種類を設定することができる。

#### CGAN生成器
条件を決めるラベルを$y$とする。  
生成器はノイズベクトル$z$とラベル$y$を使って偽のサンプル$G(z,y)=x^*|y$を生成する。  
偽のサンプルの目的は、識別器から見て与えられたラベルを持つ本物のサンプルとできるだけ近い見た目になることである。  
  
#### CGAN識別器
識別器では、
- ラベル付きの本物の組$(x,y)$
- 偽物の画像にそれを生成させるためのラベルが付いた組$(x^*|y,y)$

を受け取る。  
本物のサンプルとラベルの組では、
- 本物のサンプルをどう認識するか
- マッチする組み合わせをどう認識するか

の両方を学習する。  
生成器が生成した組では、偽のサンプルとラベルの組を認識して、本物の組とどう区別するかを学習する。  
識別器の出力は、  
入力が本物で、ラベルとマッチしているかの確信度を示す単一の確率値である。

#### 実装

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Activation, BatchNormalization, Concatenate, Dense, Embedding, Flatten, Input, Multiply, Reshape, Conv2D, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

In [3]:
img_rows = 28
img_cols = 28
channels = 1

img_shape = (img_rows, img_cols, channels)

z_dim = 100

num_classes = 10

#### 生成器
1. ラベル$y$(0～9の整数)を取り出し、z_dim次元(ランダムなノイズベクトルの長さと同じ)の密なベクトルに変換する(embedding層)。  
2. ラベル埋め込みベクトルを、ノイズベクトル$z$と組み合わせて、複合ベクトルとする(Multiply層)。
3. 出来上がったベクトルをCGANの生成器ネットワークに入力して画像を生成する。

In [14]:
def build_generator(z_dim):
    
    model = Sequential()
    
    model.add(Dense(256 * 7 * 7, input_dim=z_dim))
    model.add(Reshape((7, 7, 256)))
    
    model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    
    model.add(Conv2DTranspose(64, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    
    model.add(Conv2DTranspose(1, kernel_size=3, strides=2, padding='same'))
    
    model.add(Activation('tanh'))
    
    return model

In [25]:
def build_cgan_generator(z_dim):
    
    z = Input(shape=(z_dim,))
    
    label = Input(shape=(1,), dtype='int32')
    
    label_embedding = Embedding(num_classes, z_dim, input_length=1)(label)
    label_embedding = Flatten()(label_embedding)
    
    joined_representation = Multiply()([z, label_embedding])
    
    generator = build_generator(z_dim)
    
    conditional_img = generator(joined_representation)
    
    return Model([z, label], conditional_img)

#### 識別器
1. 0から9の整数値であるラベルを取り出し、Embedding層を使って、28×28×1=784の密ベクトルを作る
2. これを変形(Reshape)して、28×28×1の次元を持つ画像に変形する
3. 変形したラベル埋め込み画像を、対応する入力画像の上に重ねて、28×28×2の複合表現にする。
4. 画像とラベルの複合表現を、CGANの識別器ネットワークに入力する(モデルの入力次元を28×28×2とする)

In [26]:
def build_discriminator(img_shape):
    
    model = Sequential()
    
    model.add(Conv2D(64, kernel_size=3, strides=2,
                     input_shape=(img_shape[0], img_shape[1], img_shape[2]+1),
                     padding='same'))
    model.add(LeakyReLU(alpha=0.01))
    
    model.add(Conv2D(64, kernel_size=3, strides=2,
                     input_shape=img_shape,
                     padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    
    model.add(Conv2D(128, kernel_size=3, strides=2,
                     input_shape=img_shape,
                     padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [27]:
def build_cgan_discriminator(img_shape):
    
    img = Input(shape=img_shape)
    
    label = Input(shape=(1,), dtype='int32')
    
    label_embedding = Embedding(num_classes, np.prod(img_shape), input_length=1)(label)
    label_embedding = Flatten()(label_embedding)
    label_embedding = Reshape(img_shape)(label_embedding)
    
    concatenated = Concatenate(axis=-1)([img, label_embedding])
    
    discriminator = build_discriminator(img_shape)
    
    classification = discriminator(concatenated)
    
    return Model([img, label], classification)

#### モデルの構築

In [28]:
def build_cgan(generator, discriminator):
    
    z = Input(shape=(z_dim,))
    
    label = Input(shape=(1,))
    
    img = generator([z, label])
    
    classification = discriminator([img, label])
    
    # 生成器 -> 識別器と繋がる統合モデル
    # G([z, label]) = x*
    # D(x*) = 分類結果
    model = Model([z, label], classification)
    
    return model

In [29]:
discriminator = build_cgan_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

generator = build_cgan_generator(z_dim)

discriminator.trainable = False

cgan = build_cgan(generator, discriminator)
cgan.compile(loss='binary_crossentropy',
             optimizer=Adam())

#### 訓練
  
For　訓練の各反復ステップ　do  
1. 識別器の訓練  
　a. N本物のサンプルとそのラベルの組$(x,y)$からなるミニバッチを、ランダムに作る  
　b. ミニバッチに対して$D(x,y)$を計算し、逆誤差伝播によって$\theta^{(D)}$を更新することで二値分類の損失を最小化する  
　c. ランダムなノイズベクトル$z$とクラスのラベル$(z,y)$を作り、偽のサンプルからなるミニバッチを作る:$G(z,y)=x^*|y$  
　d. ミニバッチに対して$D(x^*|y,y)$を計算し、逆誤差伝播によって$\theta^{(D)}$を更新することで二値分類の損失を最小化する  
2. 生成器の訓練  
　a. ランダムなノイズベクトル$z$とクラスのラベル$(z,y)$を作り、偽のサンプルからなるミニバッチを作る:$G(z,y)=x^*|y$  
　b. ミニバッチに対して$D(x^*|y,y)$を計算し、逆誤差伝播によって$\theta^{(D)}$を更新することで二値分類の損失を最大化する  
  
End　for

In [ ]:
accuracies = []
losses = []

def train(iterations, batch_size, sample_interval):
    
    (X_train, y_train), (_,_) = mnist.load_data()
    
    X_train = X_train/127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)
    
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for iteration in range(iterations):
        # --------------------
        # 識別器の学習
        # --------------------
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs, labels = X_train[idx], y_train[idx]
        
        z = np.random.normal(0, 1, (batch_size, z_dim))
        gen_imgs = generator.predict([z, labels])
        
        d_loss_real = discriminator.train_on_batch([imgs, labels], real)
        d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # --------------------
        # 生成器の学習
        # --------------------
        z = np.random.normal(0, 1, (batch_size, z_dim))
        
        labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)
        
        g_loss = cgan.train_on_batch([z_labels], real)
        
        if (iteration + 1) % sample_interval == 0:
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (iteration + 1, d_loss[0], 100*d_loss[1], g_loss))
            
            losses.append((d_loss[0], g_loss))
            accuracies.append(100*d_loss[1])
            
            sample_images()

In [30]:
def sample_images(image_grid_rows=2, image_grid_columns=5):
    
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))
    
    labels = np.arange(0, 10).reshape(-1, 1)
    
    gen_imgs = generator.predict([z, labels])
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    fig, axs = plt.subplots(image_grid_rows,
                            image_grid_columns,
                            figsize=(10, 4),
                            sharey=True,
                            sharex=True)
    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            axs[i, j].set_title("Digit: %d" % labels[cnt])
            cnt += 1

In [ ]:
iterations = 12000
batch_size = 32
sample_interval = 1000

train(iterations, batch_size, sample_interval)